In [1]:
import os
os.chdir("/Users/aayushmalik/Documents/titanic/")

In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [301]:
titanic = pd.read_csv('train.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [423]:
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin


def clean_data(df, drop_age=True, train=True, impute_age=False, impute_fare=False):
    df.columns = df.columns.map(lambda s: s.lower())
    df = df.drop(columns=['name','ticket','cabin'])

    if drop_age:
        df = df.drop(columns='age')

    if impute_age == True:
        df.age = df.groupby(by=['pclass', 'sex'])['age'].transform(lambda x: x.fillna(x.mean()))

    if impute_fare == True:
        df.fare = df.groupby(by=['pclass', 'sex'])['fare'].transform(lambda x: x.fillna(x.mean()))
    
    if train:
        df = df.dropna()

    enc = preprocessing.OneHotEncoder(
        categories=[[1,2,3], df.sex.unique(), df.embarked.unique()]
    )
    onehot = enc.fit_transform(df[['pclass', 'sex', 'embarked']]).toarray()
    temp = pd.concat([df, pd.DataFrame(onehot, index=df.index, columns=['first', 'second', 'third', 'male', 'female', 's', 'c', 'q'])], axis=1)

    return temp.drop(columns=['sex', 'embarked', 'pclass'])


work_df = clean_data(titanic, drop_age=False, impute_age=True)
work_df.head()

,passengerid,survived,age,sibsp,parch,fare,first,second,third,male,female,s,c,q
0,1,0,22.0,1,0,7.2500,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
1,2,1,38.0,1,0,71.2833,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,3,1,26.0,0,0,7.9250,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,4,1,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,5,0,35.0,0,0,8.0500,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0


In [425]:
work_df['nfamily'] = work_df.sibsp + work_df.parch
work_df.drop(columns=['sibsp', 'parch'], inplace=True)


In [429]:
X, y = work_df.drop(columns=["passengerid", "survived"]), work_df.survived

In [459]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)


svc = svm.SVC(kernel='rbf')


In [395]:
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [443]:
# fit SVM with nested CV

from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.svm import SVC

# Number of random trials
NUM_TRIALS = 30

# Set up possible values of parameters to optimize over
p_grid = {"C": [1, 10, 50, 100], "gamma": [0.01, 0.1, 0.5, 1]}

# Array to store scores
nested_scores = np.zeros(NUM_TRIALS)

# Loop for each trial
for i in range(NUM_TRIALS):
    # Choose cross-validation techniques for the inner and outer loops,
    # independently of the dataset.
    # E.g "GroupKFold", "LeaveOneOut", "LeaveOneGroupOut", etc.
    inner_cv = KFold(n_splits=5, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=5, shuffle=True, random_state=i)

    # Nested CV with parameter optimization
    clf = GridSearchCV(estimator=svm, param_grid=p_grid, cv=inner_cv)
    
    nested_score = cross_val_score(clf, X, y, cv=outer_cv)
    nested_scores[i] = nested_score.mean()

nested_scores

array([0.79980321, 0.81324192, 0.81779344, 0.80767473, 0.80314226,
       0.81995176, 0.80549102, 0.80767473, 0.81216276, 0.81435282,
       0.81892338, 0.82336063, 0.81888529, 0.81550816, 0.80424681,
       0.8155272 , 0.80987748, 0.82113248, 0.81327366, 0.80542754,
       0.8256713 , 0.80765568, 0.80544658, 0.80204406, 0.80651939,
       0.80994096, 0.81780613, 0.81887894, 0.80647496, 0.79863518])

In [447]:
scores.mean(), scores.std()

(0.8244696969696971, 0.04182399894884093)

In [507]:
# Cross-validation for param selection

# Set up possible values of parameters to optimize over
p_grid = {"C": [0.01, 0.05, 0.1, 0.5, 1, 50, 100], "gamma": [0.01, 0.05, 0.1, 0.5, 1]}


# Choose cross-validation techniques for the inner and outer loops,
# independently of the dataset.
# E.g "GroupKFold", "LeaveOneOut", "LeaveOneGroupOut", etc.
cv = KFold(n_splits=5, shuffle=True, random_state=i)

# Nested CV with parameter optimization
clf = GridSearchCV(estimator=svc, param_grid=p_grid, cv=cv)
clf.fit(X, y)
scores[i] = clf.best_score_


In [509]:
clf.best_params_

{'C': 50, 'gamma': 0.01}

In [461]:
svc = svm.SVC(kernel='rbf', C=1, gamma=0.1)
svc.fit(X, y)

SVC(C=1, gamma=0.1)

In [463]:
test_df = pd.read_csv('test.csv')
test_df.columns = test_df.columns.map(lambda s: s.lower())

print(test_df.shape)
test_df.head()


(418, 11)


,passengerid,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [465]:
test_df = clean_data(test_df, train=False, drop_age=False, impute_age=True, impute_fare=True)
test_df

,passengerid,age,sibsp,parch,fare,first,second,third,male,female,s,c,q
0,892,34.500000,0,0,7.8292,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
1,893,47.000000,1,0,7.0000,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,894,62.000000,0,0,9.6875,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,895,27.000000,0,0,8.6625,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
4,896,22.000000,1,1,12.2875,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,24.525104,0,0,8.0500,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
414,1306,39.000000,0,0,108.9000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
415,1307,38.500000,0,0,7.2500,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
416,1308,24.525104,0,0,8.0500,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [467]:
test_df[test_df.isna().sum(axis=1) > 0]

,passengerid,age,sibsp,parch,fare,first,second,third,male,female,s,c,q


In [477]:
test_df.drop(columns=["passengerid"])

,age,sibsp,parch,fare,first,second,third,male,female,s,c,q
0,34.500000,0,0,7.8292,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
1,47.000000,1,0,7.0000,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,62.000000,0,0,9.6875,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,27.000000,0,0,8.6625,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
4,22.000000,1,1,12.2875,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
413,24.525104,0,0,8.0500,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
414,39.000000,0,0,108.9000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
415,38.500000,0,0,7.2500,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
416,24.525104,0,0,8.0500,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [479]:
test_df['nfamily'] = test_df.sibsp + test_df.parch
test_df.drop(columns=['sibsp', 'parch'], inplace=True)

In [489]:
X_test = scaler.fit_transform(test_df.drop(columns='passengerid'))

In [475]:
work_df

,passengerid,survived,age,fare,first,second,third,male,female,s,c,q,nfamily
0,1,0,22.00,7.2500,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1
1,2,1,38.00,71.2833,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1
2,3,1,26.00,7.9250,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0
3,4,1,35.00,53.1000,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1
4,5,0,35.00,8.0500,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,27.00,13.0000,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0
887,888,1,19.00,30.0000,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0
888,889,0,21.75,23.4500,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,3
889,890,1,26.00,30.0000,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0


In [371]:
drop_ids = test_df[test_df.isna().sum(axis=1) > 0].passengerid
test_df = test_df.dropna()

drop_ids

Series([], Name: passengerid, dtype: int64)

In [495]:
X

array([[-0.54925323, -0.50023975, -0.56306042, ..., -0.48271079,
        -0.30794088,  0.05785331],
       [ 0.65920553,  0.78894661,  1.77600834, ...,  2.07163382,
        -0.30794088,  0.05785331],
       [-0.24713854, -0.48664993, -0.56306042, ..., -0.48271079,
        -0.30794088, -0.5618044 ],
       ...,
       [-0.5681354 , -0.17408416, -0.56306042, ..., -0.48271079,
        -0.30794088,  1.29716872],
       [-0.24713854, -0.0422126 ,  1.77600834, ...,  2.07163382,
        -0.30794088, -0.5618044 ],
       [ 0.2060335 , -0.49017322, -0.56306042, ..., -0.48271079,
         3.24737656, -0.5618044 ]])

In [497]:
test_preds = svc.predict(X_test)

In [367]:
test_df.passengerid

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: passengerid, Length: 418, dtype: int64

In [499]:
result_df = pd.DataFrame(np.column_stack([test_df.passengerid, test_preds]), columns=['PassengerId', 'Survived'])

for id in drop_ids:
    result_df = pd.concat([result_df, pd.DataFrame({'PassengerId':[id], 'Survived':[pd.NA]})])

result_df.sort_values('PassengerId', inplace=True)

In [501]:
result_df.to_csv('svm_submission.csv',
                index=False)